In [ ]:
import pandas as pd

# Load the CSV file into a dataframe
df = pd.read_csv('0909-ori-20v0.csv', sep=',', engine='python', on_bad_lines='skip')

df.columns = ['Time', 'EventType', 'Value', 'targetx', 'targety', 'rotation', 'Col7']

# Convert the 'Time' column to a pandas datetime for easier time calculations
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S.%f')

# Initialize lists to store the results for the output CSV
trials = []
times = []
outcomes = []
targetx_list = []
targety_list = []
movement_speeds = []
rotation_speeds = []

# Initialize variables to track state within each trial
current_trial = None
trial_start_time = None
previous_position = None
previous_rotation = None

# Create a cumulative trial count for 'n' events
df['TrialCount'] = (df['EventType'] == 'n').cumsum()

# Loop over each row in the dataframe
for trial_num, trial_df in df.groupby('TrialCount'):
    trial_start_time = None
    trial_end_time = None
    targetx, targety = None, None
    trialoutcome = 0  # Default to 0 (fail) unless 'h' is found

    movement_total = 0
    rotation_total = 0
    previous_time = None
    previous_x, previous_y, previous_rotation = None, None, None

    for _, row in trial_df.iterrows():
        event_time = row['Time']
        event_type = row['EventType']
        x, y, rotation = row['targetx'], row['targety'], row['rotation']

        # Handle trial start ('n' event)
        if event_type == 'n':
            if trial_start_time is None:
                trial_start_time = event_time  # Start time of the trial

        # Handle outcome ('h' = success, 'f' = fail)
        if event_type == 'h':
            trialoutcome = 1  # Success
        elif event_type == 'f':
            trialoutcome = 0  # Fail

        # Handle target position in event 't'
        if event_type == 't':
            targetx = x
            targety = y

        # Calculate movement and rotation speed for each 'p' (position) event
        if event_type == 'p':
            if previous_time is not None:
                time_diff = (event_time - previous_time).total_seconds()
                if time_diff > 0:
                    # Calculate movement speed (distance traveled over time)
                    distance = ((x - previous_x)**2 + (y - previous_y)**2)**0.5
                    movement_speed = distance / time_diff

                    # Calculate rotation speed (change in rotation over time)
                    rotation_diff = abs(rotation - previous_rotation)
                    rotation_speed = rotation_diff / time_diff

                    # Add to totals
                    movement_total += movement_speed
                    rotation_total += rotation_speed

            # Update previous values
            previous_time = event_time
            previous_x, previous_y, previous_rotation = x, y, rotation

        # Track the trial end time when the next 'n' event starts
        if event_type == 'n' and trial_start_time is not None and trial_end_time is None:
            trial_end_time = event_time

    # If trial ends, calculate trial time and average speeds
    if trial_start_time and trial_end_time:
        trial_duration = (trial_end_time - trial_start_time).total_seconds()

        # Store the trial data
        trials.append(trial_num)
        times.append(trial_duration)
        outcomes.append(trialoutcome)
        targetx_list.append(targetx)
        targety_list.append(targety)

        # Calculate average speeds for movement and rotation
        avg_movement_speed = movement_total / len(trial_df[trial_df['EventType'] == 'p']) if len(trial_df[trial_df['EventType'] == 'p']) > 0 else 0
        avg_rotation_speed = rotation_total / len(trial_df[trial_df['EventType'] == 'p']) if len(trial_df[trial_df['EventType'] == 'p']) > 0 else 0

        movement_speeds.append(avg_movement_speed)
        rotation_speeds.append(avg_rotation_speed)

# Create a new dataframe with the collected trial data
output_df = pd.DataFrame({
    'trial': trials,
    'time': times,
    'trialoutcome': outcomes,
    'targetx': targetx_list,
    'targety': targety_list,
    'avg_movement_speed': movement_speeds,
    'avg_rotation_speed': rotation_speeds
})

# Save the results to a new CSV file
output_df.to_csv('trial_results.csv', index=False)

print("CSV file with trial data and movement/rotation speeds has been saved.")